In [1]:
# =========================
# Notebook: End-to-end cleanup + book summaries
# =========================
# What this notebook does:
# 1) Load your per-page CSV table
# 2) Sort by book_name, author, page_number (with safe filling inside gaps only)
# 3) Forward-fill title/subtitle within each book (most recent above)
# 4) Create a per-page "clean" table (pages_clean.csv)
# 5) Create a per-book summary table (books_summary.csv) with:
#    - family_type_primary (first non-unknown)
#    - family_type_set (all non-unknown seen)
#    - additive narrator symptoms + refs per symptom
#    - additive family symptoms + refs per symptom
#    - additive dx + refs per diagnosis
# 6) Print checkpoints so you can monitor as it runs

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

In [2]:
# =========================
# CONFIG (edit paths)
# =========================
# 1) Set input CSV path (your existing table)
IN_CSV = Path.home() / "Documents" / "books" / "inbox_photos" / "data_test" /"pages_clean_improved_20260129_100508.csv"


# 2) Output folder
OUT_DIR = Path.home() / "Documents" / "books" / "inbox_photos" / "data_test" / "tables_out"
OUT_DIR.mkdir(parents=True, exist_ok=True)

run_ts = datetime.now().strftime("%Y%m%d_%H%M%S")

OUT_PAGES = OUT_DIR / f"pages_clean_{run_ts}.csv"
OUT_BOOKS = OUT_DIR / f"books_summary_{run_ts}.csv"

print("IN_CSV:", IN_CSV)
print("OUT_DIR:", OUT_DIR)
print("OUT_PAGES:", OUT_PAGES.name)
print("OUT_BOOKS:", OUT_BOOKS.name)

IN_CSV: /Users/limorkissos/Documents/books/inbox_photos/data_test/pages_clean_improved_20260129_100508.csv
OUT_DIR: /Users/limorkissos/Documents/books/inbox_photos/data_test/tables_out
OUT_PAGES: pages_clean_20260204_132311.csv
OUT_BOOKS: books_summary_20260204_132311.csv


In [3]:
# =========================
# LOAD
# =========================
if not IN_CSV.exists():
    raise FileNotFoundError(f"Input CSV not found: {IN_CSV}")

df = pd.read_csv(IN_CSV)

print("Loaded rows:", len(df))
print("Loaded columns:", list(df.columns))
df.head(3)

FileNotFoundError: Input CSV not found: /Users/limorkissos/Documents/books/inbox_photos/data_test/pages_clean_improved_20260129_100508.csv

In [ ]:
# =========================
# COLUMN SETUP / SANITY
# =========================
# If you still have "title" in some versions, keep it; otherwise we focus on subtitle.
# We'll handle both title/subtitle if present.

required_cols = ["book_name", "author", "page_number", "reference", "source_file", "text"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Normalize blanks -> NaN for key text fields
for col in ["book_name", "author", "subtitle", "title", "family_type"]:
    if col in df.columns:
        df[col] = df[col].replace(r"^\s*$", np.nan, regex=True)

# Ensure page_number numeric
df["page_number"] = pd.to_numeric(df["page_number"], errors="coerce")

# Build a stable id for grouping
df["book_id"] = (
    df["book_name"].fillna("").astype(str).str.strip()
    + " | "
    + df["author"].fillna("").astype(str).str.strip()
)

print("Unique books:", df["book_id"].nunique())
print("Missing page_number:", df["page_number"].isna().sum())
print("Missing subtitle:", df["subtitle"].isna().sum() if "subtitle" in df.columns else "subtitle column not present")
print("Missing family_type:", df["family_type"].isna().sum() if "family_type" in df.columns else "family_type column not present")


NameError: name 'df' is not defined

In [ ]:
# =========================
# SORT (book -> page -> file)
# =========================
# We sort with NaN page_numbers last.
sort_cols = ["book_name", "author", "page_number", "source_file"]
for c in sort_cols:
    if c not in df.columns:
        raise ValueError(f"Expected sort column missing: {c}")

df = df.sort_values(by=sort_cols, na_position="last").reset_index(drop=True)

print("After initial sort:")
display(df[["book_name", "author", "page_number", "subtitle", "source_file"]].head(10))


After initial sort:


,book_name,author,page_number,subtitle,source_file
0,AutismInHeels,JeniferCookOtoole,90.0,NaN,PXL_20260108_040430052.jpg
1,AutismInHeels,JeniferCookOtoole,91.0,NaN,PXL_20260108_040433380.jpg
2,AutismInHeels,JeniferCookOtoole,92.0,NaN,PXL_20260108_040438580.jpg
3,AutismInHeels,JeniferCookOtoole,93.0,NaN,PXL_20260108_040442088.MP.jpg
4,AutismInHeels,JeniferCookOtoole,94.0,NaN,PXL_20260108_040446917.jpg
5,AutismInHeels,JeniferCookOtoole,95.0,NaN,PXL_20260108_040450323.jpg
6,AutismInHeels,JeniferCookOtoole,96.0,NaN,PXL_20260108_040458809.jpg
7,AutismInHeels,JeniferCookOtoole,97.0,NaN,PXL_20260108_040502277.jpg
8,AutismInHeels,JeniferCookOtoole,98.0,NaN,PXL_20260108_040506490.jpg
9,AutismInHeels,JeniferCookOtoole,100.0,Mythbusters: Open Heart Emotion,PXL_20260108_040515830.jpg


In [ ]:
# =========================
# FILL missing page_number INSIDE gaps only (per book)
# =========================
# This will only fill NaNs BETWEEN known pages (limit_area='inside').
# Leading/trailing missing pages remain NaN (no guessing).

df["page_number_original"] = df["page_number"]

def fill_pages_inside_gaps(g: pd.DataFrame) -> pd.DataFrame:
    g = g.copy()
    g["page_number_filled"] = (
        g["page_number"]
        .interpolate(method="linear", limit_area="inside")
        .round()
    )
    g["page_number_filled"] = g["page_number_filled"].astype("Int64")
    return g

df = df.groupby(["book_name", "author"], dropna=False, group_keys=False).apply(fill_pages_inside_gaps)

filled_count = df["page_number"].isna().sum() - df["page_number_filled"].isna().sum()
print("Filled page_number count (inside gaps):", int(filled_count))

# Replace page_number with filled version
df["page_number"] = df["page_number_filled"]
df.drop(columns=["page_number_filled"], inplace=True)

# Re-sort after filling (so filled pages fall into place)
df = df.sort_values(by=sort_cols, na_position="last").reset_index(drop=True)

print("After page_number fill + resort:")
display(df[["book_name", "author", "page_number_original", "page_number", "source_file"]].head(15))


Filled page_number count (inside gaps): 0
After page_number fill + resort:


/var/folders/hd/k80vq1yd4q9cs03vsc4skdkm0000gn/T/ipykernel_18060/993802424.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["book_name", "author"], dropna=False, group_keys=False).apply(fill_pages_inside_gaps)


,book_name,author,page_number_original,page_number,source_file
0,AutismInHeels,JeniferCookOtoole,90.0,90,PXL_20260108_040430052.jpg
1,AutismInHeels,JeniferCookOtoole,91.0,91,PXL_20260108_040433380.jpg
2,AutismInHeels,JeniferCookOtoole,92.0,92,PXL_20260108_040438580.jpg
3,AutismInHeels,JeniferCookOtoole,93.0,93,PXL_20260108_040442088.MP.jpg
4,AutismInHeels,JeniferCookOtoole,94.0,94,PXL_20260108_040446917.jpg
5,AutismInHeels,JeniferCookOtoole,95.0,95,PXL_20260108_040450323.jpg
6,AutismInHeels,JeniferCookOtoole,96.0,96,PXL_20260108_040458809.jpg
7,AutismInHeels,JeniferCookOtoole,97.0,97,PXL_20260108_040502277.jpg
8,AutismInHeels,JeniferCookOtoole,98.0,98,PXL_20260108_040506490.jpg
9,AutismInHeels,JeniferCookOtoole,100.0,100,PXL_20260108_040515830.jpg


In [ ]:
# =========================
# FILL titles/subtitles from most recent above (per book)
# =========================
# Forward fill within each book (book_name, author).
# We keep original columns for audit.

if "title" in df.columns:
    df["title_original"] = df["title"]
    df["title"] = df.groupby(["book_name", "author"], dropna=False)["title"].ffill()

if "subtitle" in df.columns:
    df["subtitle_original"] = df["subtitle"]
    df["subtitle"] = df.groupby(["book_name", "author"], dropna=False)["subtitle"].ffill()

print("After ffill title/subtitle:")
cols_to_show = ["book_name", "author", "page_number", "title", "subtitle", "source_file"]
cols_to_show = [c for c in cols_to_show if c in df.columns]
display(df[cols_to_show].head(20))


After ffill title/subtitle:


,book_name,author,page_number,title,subtitle,source_file
0,AutismInHeels,JeniferCookOtoole,90,REGULATION FUNCTION SKILLS,NaN,PXL_20260108_040430052.jpg
1,AutismInHeels,JeniferCookOtoole,91,REGULATION FUNCTION SKILLS,NaN,PXL_20260108_040433380.jpg
2,AutismInHeels,JeniferCookOtoole,92,AUTISM IN HEELS,NaN,PXL_20260108_040438580.jpg
3,AutismInHeels,JeniferCookOtoole,93,Trains and Fireflies,NaN,PXL_20260108_040442088.MP.jpg
4,AutismInHeels,JeniferCookOtoole,94,Trains and Fireflies,NaN,PXL_20260108_040446917.jpg
5,AutismInHeels,JeniferCookOtoole,95,Trains and Fireflies,NaN,PXL_20260108_040450323.jpg
6,AutismInHeels,JeniferCookOtoole,96,Trains and Fireflies,NaN,PXL_20260108_040458809.jpg
7,AutismInHeels,JeniferCookOtoole,97,Pivot Point,NaN,PXL_20260108_040502277.jpg
8,AutismInHeels,JeniferCookOtoole,98,Pivot Point,NaN,PXL_20260108_040506490.jpg
9,AutismInHeels,JeniferCookOtoole,100,Chapter 5,Mythbusters: Open Heart Emotion,PXL_20260108_040515830.jpg


In [ ]:
# =========================
# FAMILY TYPE: primary + set (per book) + optionally ffill for context
# =========================
# We'll create:
# - family_type_primary (first non-unknown/non-null in book, in page order)
# - family_type_set (unique non-unknown values seen across book)

# Helper: normalize family_type strings
def norm_family_type(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if not s:
        return None
    return s

if "family_type" in df.columns:
    df["family_type"] = df["family_type"].apply(norm_family_type)

    # For book primary: first non-unknown
    def first_non_unknown(series: pd.Series):
        for v in series.tolist():
            if v and v != "unknown":
                return v
        return "unknown"

    # For set: all unique non-unknown
    def union_non_unknown(series: pd.Series):
        vals = []
        for v in series.tolist():
            if v and v != "unknown":
                vals.append(v)
        vals = sorted(set(vals))
        return "|".join(vals) if vals else ""

    # Build a stable per-book table for family_type aggregates
    family_agg = (
        df.groupby(["book_name", "author"], dropna=False)
          .agg(
              family_type_primary=("family_type", first_non_unknown),
              family_type_set=("family_type", union_non_unknown),
          )
          .reset_index()
    )

    print("Family aggregates preview:")
    display(family_agg.head(10))

    # Merge family_type_primary into page df for easy viewing
    df = df.merge(family_agg, on=["book_name", "author"], how="left")
else:
    print("No family_type column present; skipping family aggregates.")


Family aggregates preview:


,book_name,author,family_type_primary,family_type_set
0,AutismInHeels,JeniferCookOtoole,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...
1,MothernessBook,JulieMGreen,chaotic_distressed,chaotic_distressed|narcissistic
2,MothernessBook,NaN,chaotic_distressed,chaotic_distressed|narcissistic


In [ ]:
# =========================
# DEFINE FLAG GROUPS (narrator, family, diagnoses)
# =========================
# We'll aggregate OR (any True) per book + store references where True.

narrator_flags = [
    "dysreg_physical_pain",
    "dysreg_dissociation_numbing",
    "dysreg_addictive_self_soothing",
    "dysreg_explosive_outbursts",
    "dysreg_people_pleasing",
    "dysreg_hyper_control_perfectionism",
    "dysreg_parentification_rescuer",
]
family_flags = [
    "fam_dysreg_physical_pain",
    "fam_dysreg_dissociation_numbing",
    "fam_dysreg_addictive_self_soothing",
    "fam_dysreg_explosive_outbursts",
    "fam_dysreg_people_pleasing",
    "fam_dysreg_hyper_control_perfectionism",
    "fam_dysreg_parentification_rescuer",
]
dx_flags = [
    "dx_autism",
    "dx_adhd",
    "dx_addictions",
    "dx_depression",
    "dx_anxiety",
    "dx_other",
]

# Keep only those that exist in df (so notebook doesn't crash if columns are missing)
narrator_flags = [c for c in narrator_flags if c in df.columns]
family_flags = [c for c in family_flags if c in df.columns]
dx_flags = [c for c in dx_flags if c in df.columns]

print("Narrator flags:", narrator_flags)
print("Family flags:", family_flags)
print("Dx flags:", dx_flags)


Narrator flags: ['dysreg_physical_pain', 'dysreg_dissociation_numbing', 'dysreg_addictive_self_soothing', 'dysreg_explosive_outbursts', 'dysreg_people_pleasing', 'dysreg_hyper_control_perfectionism', 'dysreg_parentification_rescuer']
Family flags: ['fam_dysreg_physical_pain', 'fam_dysreg_dissociation_numbing', 'fam_dysreg_addictive_self_soothing', 'fam_dysreg_explosive_outbursts', 'fam_dysreg_people_pleasing', 'fam_dysreg_hyper_control_perfectionism', 'fam_dysreg_parentification_rescuer']
Dx flags: ['dx_autism', 'dx_adhd', 'dx_addictions', 'dx_depression', 'dx_anxiety', 'dx_other']


In [ ]:
# =========================
# MAKE BOOLEAN COLUMNS SAFE
# =========================
# Some CSVs store booleans as strings ("True"/"False") -> normalize.
def to_bool(x):
    if isinstance(x, bool):
        return x
    if pd.isna(x):
        return False
    s = str(x).strip().lower()
    if s in ("true", "1", "yes", "y"):
        return True
    return False

for c in narrator_flags + family_flags + dx_flags:
    df[c] = df[c].apply(to_bool)

print("Boolean normalization done.")


Boolean normalization done.


In [ ]:
# =========================
# BUILD BOOK SUMMARY (additive flags + refs)
# =========================
# For each (book_name, author), we compute:
# - n_pages, page_min, page_max
# - family_type_primary / set
# - for each flag: <flag>_any and <flag>_refs (pipe-joined references)

def refs_where_true(g: pd.DataFrame, flag_col: str) -> str:
    refs = g.loc[g[flag_col] == True, "reference"].dropna().astype(str).tolist()
    # de-duplicate while preserving order
    seen = set()
    out = []
    for r in refs:
        if r not in seen:
            out.append(r)
            seen.add(r)
    return "|".join(out)

book_groups = df.groupby(["book_name", "author"], dropna=False)

summary_rows = []
for (book_name, author), g in book_groups:
    row = {
        "book_name": book_name,
        "author": author,
        "book_id": (str(book_name).strip() if book_name is not None else "") + " | " + (str(author).strip() if author is not None else ""),
        "n_pages": len(g),
        "page_min": int(g["page_number"].min()) if pd.notna(g["page_number"].min()) else None,
        "page_max": int(g["page_number"].max()) if pd.notna(g["page_number"].max()) else None,
    }

    # Family summary (if present)
    if "family_type_primary" in g.columns:
        row["family_type_primary"] = g["family_type_primary"].iloc[0]
    if "family_type_set" in g.columns:
        row["family_type_set"] = g["family_type_set"].iloc[0]

    # Narrator flags
    for flag in narrator_flags:
        row[f"{flag}_any"] = bool(g[flag].any())
        row[f"{flag}_refs"] = refs_where_true(g, flag)

    # Family flags
    for flag in family_flags:
        row[f"{flag}_any"] = bool(g[flag].any())
        row[f"{flag}_refs"] = refs_where_true(g, flag)

    # Diagnoses flags
    for flag in dx_flags:
        row[f"{flag}_any"] = bool(g[flag].any())
        row[f"{flag}_refs"] = refs_where_true(g, flag)

    # dx_other_label union (if present)
    if "dx_other_label" in g.columns:
        labels = g["dx_other_label"].dropna().astype(str).str.strip()
        labels = [x for x in labels.tolist() if x and x.lower() != "nan"]
        row["dx_other_label_set"] = "|".join(sorted(set(labels))) if labels else ""

    summary_rows.append(row)

books_df = pd.DataFrame(summary_rows)

# Sort book summary
books_df = books_df.sort_values(by=["book_name", "author"]).reset_index(drop=True)

print("Book summary rows:", len(books_df))
books_df.head(5)


Book summary rows: 3


,book_name,author,book_id,n_pages,page_min,page_max,family_type_primary,family_type_set,dysreg_physical_pain_any,dysreg_physical_pain_refs,dysreg_dissociation_numbing_any,dysreg_dissociation_numbing_refs,dysreg_addictive_self_soothing_any,dysreg_addictive_self_soothing_refs,dysreg_explosive_outbursts_any,dysreg_explosive_outbursts_refs,dysreg_people_pleasing_any,dysreg_people_pleasing_refs,dysreg_hyper_control_perfectionism_any,dysreg_hyper_control_perfectionism_refs,dysreg_parentification_rescuer_any,dysreg_parentification_rescuer_refs,fam_dysreg_physical_pain_any,fam_dysreg_physical_pain_refs,fam_dysreg_dissociation_numbing_any,fam_dysreg_dissociation_numbing_refs,fam_dysreg_addictive_self_soothing_any,fam_dysreg_addictive_self_soothing_refs,fam_dysreg_explosive_outbursts_any,fam_dysreg_explosive_outbursts_refs,fam_dysreg_people_pleasing_any,fam_dysreg_people_pleasing_refs,fam_dysreg_hyper_control_perfectionism_any,fam_dysreg_hyper_control_perfectionism_refs,fam_dysreg_parentification_rescuer_any,fam_dysreg_parentification_rescuer_refs,dx_autism_any,dx_autism_refs,dx_adhd_any,dx_adhd_refs,dx_addictions_any,dx_addictions_refs,dx_depression_any,dx_depression_refs,dx_anxiety_any,dx_anxiety_refs,dx_other_any,dx_other_refs,dx_other_label_set
0,AutismInHeels,JeniferCookOtoole,AutismInHeels | JeniferCookOtoole,64,90,151,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,trauma
1,MothernessBook,JulieMGreen,MothernessBook | JulieMGreen,76,13,85,chaotic_distressed,chaotic_distressed|narcissistic,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,False,,False,,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,
2,MothernessBook,NaN,MothernessBook | nan,9,5,11,chaotic_distressed,chaotic_distressed|narcissistic,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,False,,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,True,/Users/limorkissos/Documents/books/inbox_photo...,False,,False,,False,,False,,False,,False,,False,,


In [ ]:
# =========================
# PAGE VIEW: helpful monitoring columns + quick checks
# =========================
monitor_cols = [
    "book_name", "author", "page_number", "page_number_original",
    "title", "subtitle", "family_type", "family_type_primary", "family_type_set",
    "narration_voice", "source_file", "reference"
]
monitor_cols = [c for c in monitor_cols if c in df.columns]

print("Monitor columns preview (first 30 rows):")
display(df[monitor_cols].head(30))

print("\nMissing page_number AFTER fill:", df["page_number"].isna().sum())
if "subtitle" in df.columns:
    print("Missing subtitle AFTER ffill:", df["subtitle"].isna().sum())
if "family_type_primary" in df.columns:
    print("Books with family_type_primary == unknown:", (books_df.get("family_type_primary") == "unknown").sum())


Monitor columns preview (first 30 rows):


,book_name,author,page_number,page_number_original,title,subtitle,family_type,family_type_primary,family_type_set,narration_voice,source_file,reference
0,AutismInHeels,JeniferCookOtoole,90,90.0,REGULATION FUNCTION SKILLS,NaN,autistic_socially_odd,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,adult,PXL_20260108_040430052.jpg,/Users/limorkissos/Documents/books/inbox_photo...
1,AutismInHeels,JeniferCookOtoole,91,91.0,REGULATION FUNCTION SKILLS,NaN,unknown,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,adult,PXL_20260108_040433380.jpg,/Users/limorkissos/Documents/books/inbox_photo...
2,AutismInHeels,JeniferCookOtoole,92,92.0,AUTISM IN HEELS,NaN,unknown,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,adult,PXL_20260108_040438580.jpg,/Users/limorkissos/Documents/books/inbox_photo...
3,AutismInHeels,JeniferCookOtoole,93,93.0,Trains and Fireflies,NaN,unknown,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,adult,PXL_20260108_040442088.MP.jpg,/Users/limorkissos/Documents/books/inbox_photo...
4,AutismInHeels,JeniferCookOtoole,94,94.0,Trains and Fireflies,NaN,unknown,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,adult,PXL_20260108_040446917.jpg,/Users/limorkissos/Documents/books/inbox_photo...
5,AutismInHeels,JeniferCookOtoole,95,95.0,Trains and Fireflies,NaN,autistic_socially_odd,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,mixed,PXL_20260108_040450323.jpg,/Users/limorkissos/Documents/books/inbox_photo...
6,AutismInHeels,JeniferCookOtoole,96,96.0,Trains and Fireflies,NaN,unknown,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,mixed,PXL_20260108_040458809.jpg,/Users/limorkissos/Documents/books/inbox_photo...
7,AutismInHeels,JeniferCookOtoole,97,97.0,Pivot Point,NaN,unknown,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,adult,PXL_20260108_040502277.jpg,/Users/limorkissos/Documents/books/inbox_photo...
8,AutismInHeels,JeniferCookOtoole,98,98.0,Pivot Point,NaN,unknown,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,adult,PXL_20260108_040506490.jpg,/Users/limorkissos/Documents/books/inbox_photo...
9,AutismInHeels,JeniferCookOtoole,100,100.0,Chapter 5,Mythbusters: Open Heart Emotion,autistic_socially_odd,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,mixed,PXL_20260108_040515830.jpg,/Users/limorkissos/Documents/books/inbox_photo...



Missing page_number AFTER fill: 23
Missing subtitle AFTER ffill: 92
Books with family_type_primary == unknown: 0


In [ ]:
# =========================
# SAVE OUTPUTS
# =========================
df.to_csv(OUT_PAGES, index=False)
books_df.to_csv(OUT_BOOKS, index=False)

print("Saved pages table:", OUT_PAGES)
print("Saved book summary:", OUT_BOOKS)


Saved pages table: /Users/limorkissos/Documents/books/inbox_photos/data_test/tables_out/pages_clean_20260129_091019.csv
Saved book summary: /Users/limorkissos/Documents/books/inbox_photos/data_test/tables_out/books_summary_20260129_091019.csv


In [ ]:
df_clean = pd.read_csv("/Users/limorkissos/Documents/books/inbox_photos/data_test/tables_out/pages_clean_20260127_171131.csv")
df_clean.tail()

,book_name,author,page_number,subtitle,text,reference,source_file,narration_voice,family_type,dysreg_physical_pain,dysreg_dissociation_numbing,dysreg_addictive_self_soothing,dysreg_explosive_outbursts,dysreg_people_pleasing,dysreg_hyper_control_perfectionism,dysreg_parentification_rescuer,fam_dysreg_physical_pain,fam_dysreg_dissociation_numbing,fam_dysreg_addictive_self_soothing,fam_dysreg_explosive_outbursts,fam_dysreg_people_pleasing,fam_dysreg_hyper_control_perfectionism,fam_dysreg_parentification_rescuer,dx_autism,dx_adhd,dx_addictions,dx_depression,dx_anxiety,dx_other,dx_other_label,evidence_snippets,notes,book_id,page_number_original,subtitle_original,family_type_primary,family_type_set
144,MothernessBook,NaN,9.0,NaN,someone who was rude or ill-mannered. We child...,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_043743577.jpg,adult,unknown,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,"['someone who was rude or ill-mannered.', 'We ...",NaN,MothernessBook |,9.0,NaN,chaotic_distressed,chaotic_distressed|narcissistic
145,MothernessBook,NaN,10.0,NaN,"But she continued to repeat, ""It's the same ro...",/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_043748886.jpg,adult,chaotic_distressed,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,NaN,"[""I have a cough that's choking me to death"", ...",NaN,MothernessBook |,10.0,NaN,chaotic_distressed,chaotic_distressed|narcissistic
146,MothernessBook,NaN,11.0,NaN,"ever, my grandmother's father was not to be fo...",/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_043751299.jpg,mixed,chaotic_distressed,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,NaN,['In this house you make a bordello out of eve...,NaN,MothernessBook |,11.0,NaN,chaotic_distressed,chaotic_distressed|narcissistic
147,MothernessBook,NaN,NaN,NATALIA GINZBURG,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_043717858.jpg,unclear,unknown,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,[],No text extracted; labeling skipped.,MothernessBook |,NaN,NATALIA GINZBURG,chaotic_distressed,chaotic_distressed|narcissistic
148,MothernessBook,NaN,NaN,NATALIA GINZBURG,"ever, my grandmother’s father was not to be fo...",/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_043751420.jpg,adult,chaotic_distressed,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,NaN,"['my grandmother always said, meaning that for...",NaN,MothernessBook |,NaN,NaN,chaotic_distressed,chaotic_distressed|narcissistic


In [ ]:
df_clean[df_clean["book_name"].str.contains("Mother", na=False)][
    ["book_name", "author"]
].value_counts()

book_name       author     
MothernessBook  JulieMGreen    76
Name: count, dtype: int64

In [ ]:
'''
# --- define correction ---
OLD_BOOK_NAME = "MothernessBook"
NEW_BOOK_NAME = "Family Lexicon"
NEW_AUTHOR = "Natalia Ginzburg"

mask = df_clean["book_name"] == OLD_BOOK_NAME

print("Rows to fix:", mask.sum())

# Apply fix
df_clean.loc[mask, "book_name"] = NEW_BOOK_NAME
df_clean.loc[mask, "author"] = NEW_AUTHOR

# Sanity check
print(
    df_clean.loc[mask, ["book_name", "author"]]
    .drop_duplicates()
)
'''

Rows to fix: 85
         book_name            author
64  Family Lexicon  Natalia Ginzburg


In [ ]:
from datetime import datetime

ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_path = OUT_DIR / f"jsontable_corrected_{ts}.csv"

df_clean.to_csv(out_path, index=False)
print("Saved corrected table:", out_path)

Saved corrected table: /Users/limorkissos/Documents/books/inbox_photos/data_test/tables_out/jsontable_corrected_20260129_091019.csv


In [ ]:
df_table_natalia = pd.read_csv("/Users/limorkissos/Documents/books/inbox_photos/data_test/tables_out/jsontable_corrected_20260127_172215.csv")
df_table_natalia.shape
df_table_natalia.head()

,book_name,author,page_number,subtitle,text,reference,source_file,narration_voice,family_type,dysreg_physical_pain,dysreg_dissociation_numbing,dysreg_addictive_self_soothing,dysreg_explosive_outbursts,dysreg_people_pleasing,dysreg_hyper_control_perfectionism,dysreg_parentification_rescuer,fam_dysreg_physical_pain,fam_dysreg_dissociation_numbing,fam_dysreg_addictive_self_soothing,fam_dysreg_explosive_outbursts,fam_dysreg_people_pleasing,fam_dysreg_hyper_control_perfectionism,fam_dysreg_parentification_rescuer,dx_autism,dx_adhd,dx_addictions,dx_depression,dx_anxiety,dx_other,dx_other_label,evidence_snippets,notes,book_id,page_number_original,subtitle_original,family_type_primary,family_type_set
0,AutismInHeels,JeniferCookOtoole,90.0,NaN,Challenging to Spectrum Minds Because: We have...,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040430052.jpg,adult,autistic_socially_odd,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,NaN,['I wrote an entire essay on why the essay que...,NaN,AutismInHeels | JeniferCookOtoole,90.0,NaN,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...
1,AutismInHeels,JeniferCookOtoole,91.0,NaN,Initiation of Action involves beginning any ne...,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040433380.jpg,adult,unknown,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,NaN,['Our minds are more prone to zooming in inten...,NaN,AutismInHeels | JeniferCookOtoole,91.0,NaN,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...
2,AutismInHeels,JeniferCookOtoole,92.0,NaN,"Other EF Skills Involved: attention, cognitive...",/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040438580.jpg,adult,unknown,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,[],NaN,AutismInHeels | JeniferCookOtoole,92.0,NaN,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...
3,AutismInHeels,JeniferCookOtoole,93.0,NaN,"Thanks to EF, typical minds can readily set an...",/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040442088.MP.jpg,adult,unknown,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,NaN,['The calling card of autistic brains—the sing...,NaN,AutismInHeels | JeniferCookOtoole,93.0,NaN,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...
4,AutismInHeels,JeniferCookOtoole,94.0,NaN,"Trains: If typical minds are cars, making shar...",/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040446917.jpg,adult,unknown,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,[],NaN,AutismInHeels | JeniferCookOtoole,94.0,NaN,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...


In [ ]:

# fix missing subtitle - copy from json to the table

import json
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# df should already be loaded from your jsontable_*.csv

''''
# --- 1) choose the author/book filter ---
# Adjust this if your author string differs (e.g., "JeniferCookOtoole")
AUTHOR_MATCH = "Jen"  # broad
BOOK_MATCH = "AutismInHeels"  # optional; set to "" to ignore

df_table_natalia["author"] = df_table_natalia["author"].astype(str)
df_table_natalia["book_name"] = df_table_natalia["book_name"].astype(str)

mask_book = df_table_natalia["book_name"].str.contains(BOOK_MATCH, case=False, na=False) if BOOK_MATCH else True
mask_author = df_table_natalia["author"].str.contains(AUTHOR_MATCH, case=False, na=False)

jdf = df_table_natalia[mask_book & mask_author].copy()

print("Rows in Jennifer book subset:", len(jdf))
display(jdf[["book_name","author"]].drop_duplicates().head(20))

# --- 2) find rows with missing subtitle in the TABLE ---
def is_missing_subtitle(x):
    if pd.isna(x):
        return True
    s = str(x).strip()
    return s == "" or s.lower() == "nan"

missing_mask = jdf["subtitle"].apply(is_missing_subtitle)
missing_idx = jdf.index[missing_mask].tolist()

print("Rows with missing subtitle (in table):", len(missing_idx))
display(df.loc[missing_idx, ["book_name","author","page_number","subtitle","reference","source_file"]].head(20))
'''
# --- 3) helper: from reference -> sidecar json path ---
def sidecar_json_path(reference_value: str) -> Path:
    """
    Your reference is the image path (e.g., .../PXL_....jpg).
    Sidecar is: image.jpg.json (same folder).
    """
    img_path = Path(reference_value)
    return img_path.with_suffix(img_path.suffix + ".json")

# --- 4) pull subtitle from JSON sidecar and update df ---
updated = 0
not_found = 0
no_subtitle_in_json = 0
errors = 0

# optional: also pull title if you want
PULL_TITLE_TOO = False

for i in missing_idx:
    ref = df.at[i, "reference"]
    if pd.isna(ref) or str(ref).strip() == "":
        errors += 1
        continue

    jp = sidecar_json_path(str(ref))

    if not jp.exists():
        not_found += 1
        continue

    try:
        data = json.loads(jp.read_text(encoding="utf-8"))

        # read subtitle from JSON (handle alt key names if needed)
        subtitle = data.get("subtitle")
        if subtitle is None:
            # sometimes people accidentally wrote page_subtitle etc.
            subtitle = data.get("page_subtitle")

        if subtitle is None or str(subtitle).strip() == "":
            no_subtitle_in_json += 1
            continue

        df.at[i, "subtitle"] = str(subtitle).strip()

        if PULL_TITLE_TOO and ("title" in df.columns):
            title = data.get("title")
            if title and str(title).strip():
                df.at[i, "title"] = str(title).strip()

        updated += 1

    except Exception as e:
        errors += 1

print("\n--- Update summary ---")
print("Updated subtitles:", updated)
print("JSON sidecar not found:", not_found)
print("JSON found but subtitle missing:", no_subtitle_in_json)
print("Errors:", errors)

# --- 5) verify remaining missing subtitles for this subset ---
jdf2 = df[mask_book & mask_author].copy()
missing_after = jdf2["subtitle"].apply(is_missing_subtitle).sum()
print("Missing subtitles AFTER update (Jennifer subset):", int(missing_after))

# --- 6) save corrected table ---
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_path = Path.home() / "Documents" / "books" / "inbox_photos" / "data_test" / f"jsontable_with_subtitles_fixed_{ts}.csv"
df.to_csv(out_path, index=False)
print("Saved corrected table:", out_path)

# Preview a few fixed rows
fixed_preview = df.loc[missing_idx, ["page_number","subtitle","reference","source_file"]].head(30)
display(fixed_preview)


Rows in Jennifer book subset: 64


,book_name,author
0,AutismInHeels,JeniferCookOtoole


Rows with missing subtitle (in table): 9


,book_name,author,page_number,subtitle,reference,source_file
0,AutismInHeels,JeniferCookOtoole,90,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040430052.jpg
1,AutismInHeels,JeniferCookOtoole,91,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040433380.jpg
2,AutismInHeels,JeniferCookOtoole,92,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040438580.jpg
3,AutismInHeels,JeniferCookOtoole,93,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040442088.MP.jpg
4,AutismInHeels,JeniferCookOtoole,94,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040446917.jpg
5,AutismInHeels,JeniferCookOtoole,95,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040450323.jpg
6,AutismInHeels,JeniferCookOtoole,96,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040458809.jpg
7,AutismInHeels,JeniferCookOtoole,97,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040502277.jpg
8,AutismInHeels,JeniferCookOtoole,98,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040506490.jpg



--- Update summary ---
Updated subtitles: 0
JSON sidecar not found: 0
JSON found but subtitle missing: 9
Errors: 0
Missing subtitles AFTER update (Jennifer subset): 9
Saved corrected table: /Users/limorkissos/Documents/books/inbox_photos/data_test/jsontable_with_subtitles_fixed_20260129_091019.csv


,page_number,subtitle,reference,source_file
0,90,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040430052.jpg
1,91,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040433380.jpg
2,92,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040438580.jpg
3,93,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040442088.MP.jpg
4,94,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040446917.jpg
5,95,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040450323.jpg
6,96,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040458809.jpg
7,97,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040502277.jpg
8,98,NaN,/Users/limorkissos/Documents/books/inbox_photo...,PXL_20260108_040506490.jpg


In [ ]:
''''
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np

# --- Load (uncomment + set if needed) ---
# BOOKS_SUMMARY_CSV = Path("/FULL/PATH/TO/books_summary_....csv")
# books_df = pd.read_csv(BOOKS_SUMMARY_CSV)

OLD_BOOK_NAME = "MothernessBook"
NEW_BOOK_NAME = "Family Lexicon"
NEW_AUTHOR = "Natalia Ginzburg"

# --- Choose output folder robustly ---
# If you loaded from a CSV, save next to it. Otherwise, use OUT_DIR if you already defined it, else current folder.
if "BOOKS_SUMMARY_CSV" in globals():
    OUT_DIR = Path(BOOKS_SUMMARY_CSV).parent
else:
    OUT_DIR = Path(globals().get("OUT_DIR", Path.cwd()))

# --- 1) Fix rows (book_name + author) ---
mask = (books_df["book_name"].astype(str) == OLD_BOOK_NAME)

print("Rows with MothernessBook:", int(mask.sum()))
if int(mask.sum()) == 0:
    print("No rows found with MothernessBook in book_name. Will still rebuild book_id + de-dup.")
else:
    print("Before (sample):")
    print(books_df.loc[mask, ["book_name", "author", "book_id"]].head(10))

    books_df.loc[mask, "book_name"] = NEW_BOOK_NAME
    books_df.loc[mask, "author"] = NEW_AUTHOR

# --- 2) Rebuild book_id from (book_name | author) ---
# (This is the key fix that removes the lingering MothernessBook | ... values)
books_df["book_name"] = books_df["book_name"].astype(str).str.strip()
books_df["author"] = books_df["author"].astype(str).str.strip()

# If author was missing/NaN in some rows, make it empty string before building the id
books_df["author"] = books_df["author"].replace({"nan": "", "None": ""})

books_df["book_id"] = books_df["book_name"] + " | " + books_df["author"]

# --- 3) Collapse duplicates to guarantee 1 row per book_id ---
# - numeric columns: sum (counts/pages/etc.)
# - non-numeric: take first non-empty
def first_non_empty(series: pd.Series):
    s = series.dropna().astype(str)
    s = s[s.str.strip() != ""]
    return s.iloc[0] if len(s) else np.nan

numeric_cols = books_df.select_dtypes(include=["number"]).columns.tolist()
non_numeric_cols = [c for c in books_df.columns if c not in numeric_cols]

agg = {c: "sum" for c in numeric_cols}
agg.update({c: first_non_empty for c in non_numeric_cols})

books_df = (
    books_df
    .groupby("book_id", as_index=False)
    .agg(agg)
)

# --- 4) Quick sanity checks ---
print("\nDistinct book_name:", books_df["book_name"].nunique())
print("Distinct book_id:", books_df["book_id"].nunique())
print("\nFinal books:")
print(books_df[["book_name", "author", "book_id"]])

# --- 5) Save ---
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_path = OUT_DIR / f"books_summary_corrected_{ts}.csv"
books_df.to_csv(out_path, index=False)
print("\nSaved corrected book summary:", out_path)
'''

Rows with MothernessBook: 0
No rows found with MothernessBook in book_name. Will still rebuild book_id + de-dup.

Distinct book_name: 2
Distinct book_id: 2

Final books:
        book_name             author                            book_id
0   AutismInHeels  JeniferCookOtoole  AutismInHeels | JeniferCookOtoole
1  Family Lexicon   Natalia Ginzburg  Family Lexicon | Natalia Ginzburg

Saved corrected book summary: /Users/limorkissos/Documents/books/inbox_photos/data_test/tables_out/books_summary_corrected_20260129_093156.csv


In [ ]:
# =========================
# OPTIONAL: quick interactive helpers (edit/use freely)
# =========================

# 1) View one book quickly
def show_book(book_name_contains: str):
    mask = df["book_name"].fillna("").str.contains(book_name_contains, case=False, na=False)
    sub = df.loc[mask].sort_values(by=["author", "page_number"], na_position="last")
    cols = ["book_name", "author", "page_number", "title", "subtitle", "family_type", "narration_voice", "source_file"]
    cols = [c for c in cols if c in sub.columns]
    print("Rows:", len(sub))
    display(sub[cols].head(80))

# 2) See which pages triggered a flag (narrator or family)
def show_flag_pages(flag_col: str, book_name_contains: str = ""):
    if flag_col not in df.columns:
        raise ValueError(f"Flag not found: {flag_col}")
    sub = df.copy()
    if book_name_contains:
        sub = sub[sub["book_name"].fillna("").str.contains(book_name_contains, case=False, na=False)]
    sub = sub[sub[flag_col] == True]
    cols = ["book_name", "author", "page_number", "subtitle", flag_col, "reference"]
    cols = [c for c in cols if c in sub.columns]
    print("Rows:", len(sub))
    display(sub[cols].head(200))

print("Helpers ready: show_book('Family') , show_flag_pages('fam_dysreg_explosive_outbursts', 'Family')")


Helpers ready: show_book('Family') , show_flag_pages('fam_dysreg_explosive_outbursts', 'Family')


In [ ]:
df_5_50 = pd.read_csv("/Users/limorkissos/Documents/books/inbox_photos/data_test/tables_out/books_summary_corrected_20260127_174722.csv")
df_5_50.head()

,book_name,author,book_id,n_pages,page_min,page_max,family_type_primary,family_type_set,dysreg_physical_pain_any,dysreg_physical_pain_refs,dysreg_dissociation_numbing_any,dysreg_dissociation_numbing_refs,dysreg_addictive_self_soothing_any,dysreg_addictive_self_soothing_refs,dysreg_explosive_outbursts_any,dysreg_explosive_outbursts_refs,dysreg_people_pleasing_any,dysreg_people_pleasing_refs,dysreg_hyper_control_perfectionism_any,dysreg_hyper_control_perfectionism_refs,dysreg_parentification_rescuer_any,dysreg_parentification_rescuer_refs,fam_dysreg_physical_pain_any,fam_dysreg_physical_pain_refs,fam_dysreg_dissociation_numbing_any,fam_dysreg_dissociation_numbing_refs,fam_dysreg_addictive_self_soothing_any,fam_dysreg_addictive_self_soothing_refs,fam_dysreg_explosive_outbursts_any,fam_dysreg_explosive_outbursts_refs,fam_dysreg_people_pleasing_any,fam_dysreg_people_pleasing_refs,fam_dysreg_hyper_control_perfectionism_any,fam_dysreg_hyper_control_perfectionism_refs,fam_dysreg_parentification_rescuer_any,fam_dysreg_parentification_rescuer_refs,dx_autism_any,dx_autism_refs,dx_adhd_any,dx_adhd_refs,dx_addictions_any,dx_addictions_refs,dx_depression_any,dx_depression_refs,dx_anxiety_any,dx_anxiety_refs,dx_other_any,dx_other_refs,dx_other_label_set
0,AutismInHeels,JeniferCookOtoole,AutismInHeels | JeniferCookOtoole,64,90,151,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,trauma
1,Family Lexicon,Natalia Ginzburg,MothernessBook | JulieMGreen,76,13,85,chaotic_distressed,chaotic_distressed|narcissistic,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,False,NaN,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,NaN
2,Family Lexicon,Natalia Ginzburg,MothernessBook | nan,9,5,11,chaotic_distressed,chaotic_distressed|narcissistic,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,NaN


In [ ]:
df_5_50

,book_name,author,book_id,n_pages,page_min,page_max,family_type_primary,family_type_set,dysreg_physical_pain_any,dysreg_physical_pain_refs,dysreg_dissociation_numbing_any,dysreg_dissociation_numbing_refs,dysreg_addictive_self_soothing_any,dysreg_addictive_self_soothing_refs,dysreg_explosive_outbursts_any,dysreg_explosive_outbursts_refs,dysreg_people_pleasing_any,dysreg_people_pleasing_refs,dysreg_hyper_control_perfectionism_any,dysreg_hyper_control_perfectionism_refs,dysreg_parentification_rescuer_any,dysreg_parentification_rescuer_refs,fam_dysreg_physical_pain_any,fam_dysreg_physical_pain_refs,fam_dysreg_dissociation_numbing_any,fam_dysreg_dissociation_numbing_refs,fam_dysreg_addictive_self_soothing_any,fam_dysreg_addictive_self_soothing_refs,fam_dysreg_explosive_outbursts_any,fam_dysreg_explosive_outbursts_refs,fam_dysreg_people_pleasing_any,fam_dysreg_people_pleasing_refs,fam_dysreg_hyper_control_perfectionism_any,fam_dysreg_hyper_control_perfectionism_refs,fam_dysreg_parentification_rescuer_any,fam_dysreg_parentification_rescuer_refs,dx_autism_any,dx_autism_refs,dx_adhd_any,dx_adhd_refs,dx_addictions_any,dx_addictions_refs,dx_depression_any,dx_depression_refs,dx_anxiety_any,dx_anxiety_refs,dx_other_any,dx_other_refs,dx_other_label_set
0,AutismInHeels,JeniferCookOtoole,AutismInHeels | JeniferCookOtoole,64,90,151,autistic_socially_odd,alcoholic_addicted|autistic_socially_odd|chaot...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,trauma
1,Family Lexicon,Natalia Ginzburg,MothernessBook | JulieMGreen,76,13,85,chaotic_distressed,chaotic_distressed|narcissistic,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,False,NaN,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,NaN
2,Family Lexicon,Natalia Ginzburg,MothernessBook | nan,9,5,11,chaotic_distressed,chaotic_distressed|narcissistic,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,True,/Users/limorkissos/Documents/books/inbox_photo...,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,NaN
